In [ ]:
# Name: Wenqi Wang
# Github username: acse-ww721

In [24]:
# Basic setting for Jupyter_notebook to import utils
import os
import sys

notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, "../../"))

sys.path.append(project_root)

In [25]:
import cdsapi
import threading
import time
import os
import sys
from tqdm import tqdm
# from utils import time_utils, folder_utils
from concurrent.futures import ThreadPoolExecutor  # thread pool module

In [26]:
def get_current_directory():
    if "__file__" in globals():
        # Running in a Python file
        return os.path.abspath(os.path.dirname(__file__))
    else:
        # Running in a Jupyter Notebook
        return os.path.abspath(os.path.dirname(""))


def find_folder(c, data_folder, data_category, output_folder):
    # c: country list
    current_directory = get_current_directory()
    project_root = os.path.abspath(os.path.join(current_directory, "..", "."))
    folder_name = f"{c}_{output_folder}"
    folder_path = os.path.join(
        project_root, data_folder, data_category, output_folder, folder_name
    )

    return folder_path


def create_folder(c, data_folder, data_category, output_folder):
    # c: country list
    folder_path = find_folder(c, data_folder, data_category, output_folder)

    try:
        os.makedirs(folder_path, exist_ok=True)
        print(f"Folder '{folder_path}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_path}' already exists.")

    return folder_path

In [27]:

def time_select(df, date_column, start_date, end_date):
    df[date_column] = pd.to_datetime(df[date_column], format="%Y%m%d")
    filtered_df = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]
    return filtered_df


# Example
# filtered_noaa_df = time_select(processed_df_noaa, "DATE", start_date, end_date)
# filtered_asos_df = time_select(processed_df_asos, "date", start_date, end_date)


def is_leap_year(year):
    year = int(year)  # Convert the string to integer
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)


def days_check(year, month):
    year = int(year)  # Convert the string to integer
    days_by_month = {
        "01": 31,  # January
        "02": 29 if is_leap_year(year) else 28,  # February
        "03": 31,  # March
        "04": 30,  # April
        "05": 31,  # May
        "06": 30,  # June
        "07": 31,  # July
        "08": 31,  # August
        "09": 30,  # September
        "10": 31,  # October
        "11": 30,  # November
        "12": 31,  # December
    }

    return [str(day).zfill(2) for day in range(1, days_by_month.get(month, 0) + 1)]

In [28]:
# folder setting
country = "GB"

data_folder = "data"
data_category = "raw_data"
output_folder = "ERA5_DATA"

In [23]:
input = create_folder(country,data_folder,data_category,output_folder)
input

Folder 'F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA' created successfully.


'F:\\JuPyterNotebook\\data\\raw_data\\ERA5_DATA\\GB_ERA5_DATA'

In [29]:
# variable setting
data_year = [  # the target years
    "1979",
    "1980",
    "1981",
    "1982",
    "1983",
    "1984",
    "1985",
    "1986",
    "1987",
    "1988",
    "1989",
    "1990",
    "1991",
    "1992",
    "1993",
    "1994",
    "1995",
    "1996",
    "1997",
    "1998",
    "1999",
    "2000",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

data_month = [  # the target months
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
]

data_day = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
    "17",
    "18",
    "19",
    "20",
    "21",
    "22",
    "23",
    "24",
    "25",
    "26",
    "27",
    "28",
    "29",
    "30",
    "31",
]  # the target day  # s


data_time = [  # the target times_UTC
    "00:00",
    "01:00",
    "02:00",
    "03:00",
    "04:00",
    "05:00",
    "06:00",
    "07:00",
    "08:00",
    "09:00",
    "10:00",
    "11:00",
    "12:00",
    "13:00",
    "14:00",
    "15:00",
    "16:00",
    "17:00",
    "18:00",
    "19:00",
    "20:00",
    "21:00",
    "22:00",
    "23:00",
]

In [30]:
# 1,571,328 > 120,000 per time

# dataset setting
dataset = "reanalysis-era5-pressure-levels"
variable_list = [
    "temperature", # priority download T850
    # "relative_humidity",
    # 'geopotential', the common variable for weather forecasting
    # "specific_humidity",
]

pressure_level = "850"


In [31]:
area_uk = [
    58,
    -7,
    50,
    2,
]

In [47]:
def era5_get_data_t850(c, dataset, variable_list, year, month):
    # download the whole year data
    try:
        output_directory = create_folder(
            country, data_folder, data_category, output_folder
        )
        output_filename = f"era5_pressure_level_{year}_{month}_{pressure_level}.nc"
        output_filepath = os.path.join(output_directory, output_filename)
        c.retrieve(
            dataset,
            {
                "product_type": "reanalysis",
                "format": "netcdf",
                "variable":'temperature',
                "pressure_level": '850',
                "year": year,
                "month": month,
                "day": days_check(year, month),
                "time": data_time,
                "area": area_uk,
            },
            output_filepath,
        )

        print(f"{output_filename} done!")

    except Exception as e:
        print(f"Error downloading {output_filename}: {e}\n")


In [39]:
# 44*12 =528

In [45]:
def thread_function(year, month):
    c = cdsapi.Client()  # Initialize client within the thread

    start_time = time.time()  # Record start time
    era5_get_data_t850(
        c,
        dataset,
        variable_list,
        year,
        month,
    )
    end_time = time.time()  # Record end time
    run_time = end_time - start_time
    print(f"Download time: {run_time:.3f} s")


In [44]:
# Create a thread pool  # 8 threads
with ThreadPoolExecutor(max_workers=2) as executor:
    # iterate through the data_year and data month to be the most efficient
    for i in data_year:
        for k in data_month:
            executor.submit(thread_function, i, k)

In [48]:
for i in data_year:
    for k in data_month:
       thread_function(i, k)

2023-08-26 12:48:08,016 INFO Welcome to the CDS
2023-08-26 12:48:08,016 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


Folder 'F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA' created successfully.


2023-08-26 12:48:08,087 INFO Request is queued
2023-08-26 12:48:09,139 INFO Request is running
2023-08-26 12:48:57,704 INFO Request is completed
2023-08-26 12:48:57,706 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data6/adaptor.mars.internal-1693050523.9571652-30807-11-9f2e4cee-f5dd-4cb7-bd14-d8cfa9322780.nc to F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA\era5_pressure_level_1979_01_850.nc (1.7M)
2023-08-26 12:48:58,194 INFO Download rate 3.6M/s                                                                      
2023-08-26 12:48:58,280 INFO Welcome to the CDS
2023-08-26 12:48:58,281 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-08-26 12:48:58,357 INFO Request is queued


era5_pressure_level_1979_01_850.nc done!
Download time: 50.386 s
Folder 'F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA' created successfully.


2023-08-26 12:48:59,398 INFO Request is running
2023-08-26 12:49:47,979 INFO Request is completed
2023-08-26 12:49:47,979 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data2/adaptor.mars.internal-1693050574.4731333-15978-19-7f221243-4610-42b1-919f-0423ea92b380.nc to F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA\era5_pressure_level_1979_02_850.nc (1.6M)
2023-08-26 12:49:48,475 INFO Download rate 3.2M/s                                                                      
2023-08-26 12:49:48,558 INFO Welcome to the CDS
2023-08-26 12:49:48,560 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-08-26 12:49:48,633 INFO Request is queued


...

era5_pressure_level_2022_10_850.nc done!
Download time: 50.432 s
Folder 'F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA' created successfully.


2023-08-26 20:10:36,549 INFO Request is running
2023-08-26 20:11:25,083 INFO Request is completed
2023-08-26 20:11:25,083 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1693077071.1071901-15226-14-1258ecb5-30ab-4ada-a320-2cc86c6b6985.nc to F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA\era5_pressure_level_2022_11_850.nc (1.7M)
2023-08-26 20:11:25,549 INFO Download rate 3.6M/s                                                                      
2023-08-26 20:11:25,670 INFO Welcome to the CDS
2023-08-26 20:11:25,670 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-08-26 20:11:25,747 INFO Request is queued


era5_pressure_level_2022_11_850.nc done!
Download time: 50.245 s
Folder 'F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA' created successfully.


2023-08-26 20:11:26,786 INFO Request is running
2023-08-26 20:12:15,344 INFO Request is completed
2023-08-26 20:12:15,344 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data1/adaptor.mars.internal-1693077119.5366566-12224-7-5dccc770-1670-4ff4-8231-0ece4686efd8.nc to F:\JuPyterNotebook\data\raw_data\ERA5_DATA\GB_ERA5_DATA\era5_pressure_level_2022_12_850.nc (1.7M)
2023-08-26 20:12:15,826 INFO Download rate 3.7M/s                                                                      


era5_pressure_level_2022_12_850.nc done!
Download time: 50.239 s


In [42]:
# TEST CDS API LINK
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': 'temperature',
        'pressure_level': '850',
        'year': '2021',
        'month': '01',
        'day': '01',
        'time': '00:00',
        'area': [
            58, -7, 50,
            1.75,
        ],
    },
    'download.nc')

2023-08-26 12:39:52,479 INFO Welcome to the CDS
2023-08-26 12:39:52,479 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-08-26 12:39:52,624 INFO Request is completed
2023-08-26 12:39:52,625 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data7/adaptor.mars.internal-1693049886.692061-15368-8-2ca281e4-d5a0-429e-b947-f2b2c9eb8759.nc to download.nc (3.7K)
2023-08-26 12:39:52,807 INFO Download rate 20.3K/s                                                                     


Result(content_length=3764,content_type=application/x-netcdf,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data7/adaptor.mars.internal-1693049886.692061-15368-8-2ca281e4-d5a0-429e-b947-f2b2c9eb8759.nc)